In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of a UNI v2 LUNC/BUSD LP strategy
---


In [2]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

In [3]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure, plot_price_ratio

mkt_pair = 'LUNC/BUSD'
start, end = '2022-10-01 00:00:00', '2023-04-25 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

qty_factor = 0.001
df_trades = client.get_trade_data(mkt_pair, frequency, start, end)
df_trades["quantity"] = qty_factor*df_trades["quantity"]
show(Div(text=format_df(df_trades.head(10), width=800)))

In [4]:
from terra_algo_backtest.market import MarketQuote, MarketPair, new_market, Pool, with_mkt_price
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.strategy import get_strategy

liquidity_usd = 1000000
# LUNC/BUSD market price
base = MarketQuote("LUNC/USD", df_trades.price.iloc[0])
# LUNC/BUSD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 100000 USD market for LUNC/USTC
mkt = new_market(liquidity_usd, quote, base, 0.003)
# load strategy 
strategy = get_strategy("uni_v2")
# run simulation
simul = swap_simulation(mkt, df_trades, strategy)
# display sim results
show(new_simulation_figure(mkt, simul))
# show(Div(text=format_df(simul["breakdown"].head(100), width=3200)))

Function 'trade_summary' executed in 0.0133s
Function 'sim_results' executed in 0.0746s
Function 'swap_simulation' executed in 0.5052s


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

simul["breakdown"].head(100)

,side,arb_profit,price,price_impact,mid_price,mkt_price,spread,avg_price,current_base,current_quote,cp_invariant,total_fees_paid_quote,total_volume_base,total_volume_quote,asset_base_pct,hold_portfolio,current_portfolio,trade_pnl,total_pnl,roi,impermanent_loss,mkt_price_ratio,volume_base,volume_quote,fees_paid_quote,trade_pnl_pct,fees_pnl_pct,total_arb_profit
trade_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-10-01 00:00:00,sell,0.00000,0.00030,0.00268,0.00030,0.00030,0.00000,0.00030,"1,657,881,818.18767","498,660.13829","826,719,576,719,576.87500",4.03168,"4,442,664.74851","-1,343.89339",0.50000,"997,327.89871","997,320.27657",-7.62214,-3.59046,-0.00000,0.00000,1.00000,"4,442,664.74851","-1,343.89339",4.03168,-0.00001,0.00000,0.00000
2022-10-01 01:00:00,sell,1.08584,0.00030,0.00539,0.00030,0.00030,-0.00000,0.00030,"1,666,861,106.23522","495,973.88386","826,719,576,719,576.75000",12.11469,"13,421,952.79606","-4,038.23083",0.50000,"991,992.30164","991,947.76773",-44.53391,-32.41922,-0.00003,-0.00003,0.98393,"8,979,288.04755","-2,694.33744",8.08301,-0.00004,0.00001,1.08584
2022-10-01 01:00:00,sell,0.00000,0.00030,0.00171,0.00030,0.00030,0.00000,0.00030,"1,669,712,883.20474","495,126.78799","826,719,576,719,576.75000",14.66363,"16,273,729.76559","-4,887.87564",0.50000,"990,315.73599","990,253.57597",-62.16002,-47.49639,-0.00005,-0.00003,0.98393,"2,851,776.96952",-849.64481,2.54893,-0.00006,0.00001,1.08584
2022-10-01 02:00:00,buy,0.00000,0.00030,-0.00044,0.00030,0.00030,0.00000,0.00030,"1,668,971,531.04916","495,346.72182","826,719,576,719,576.75000",15.32541,"15,532,377.61001","-4,667.28002",0.50000,"990,750.75148","990,693.44364",-57.30784,-41.98242,-0.00004,-0.00001,0.99011,"-741,352.15557",220.59562,0.66179,-0.00006,0.00002,1.08584
2022-10-01 03:00:00,buy,0.00000,0.00030,-0.00014,0.00030,0.00030,0.00000,0.00030,"1,668,732,265.12883","495,417.74555","826,719,576,719,576.75000",15.53913,"15,293,111.68967","-4,596.04258",0.50000,"990,891.27334","990,835.49110",-55.78224,-40.24311,-0.00004,-0.00002,0.98647,"-239,265.92034",71.23744,0.21371,-0.00006,0.00002,1.08584
2022-10-01 04:00:00,sell,0.85416,0.00030,0.00531,0.00029,0.00029,-0.00000,0.00030,"1,677,637,180.62909","492,788.06304","826,719,576,719,576.75000",23.45191,"24,198,027.18993","-7,233.63788",0.50000,"985,701.85106","985,576.12607",-125.72498,-102.27307,-0.00010,-0.00011,0.97133,"8,904,915.50026","-2,637.59530",7.91279,-0.00013,0.00002,1.94000
2022-10-01 04:00:00,sell,0.00000,0.00029,0.00048,0.00029,0.00029,0.00000,0.00030,"1,678,443,513.96211","492,551.32499","826,719,576,719,576.75000",24.16426,"25,004,360.52295","-7,471.08828",0.50000,"985,236.02876","985,102.64998",-133.37878,-109.21452,-0.00011,-0.00011,0.97133,"806,333.33302",-237.45040,0.71235,-0.00014,0.00002,1.94000
2022-10-01 05:00:00,sell,0.00000,0.00029,0.00097,0.00029,0.00029,-0.00000,0.00030,"1,680,071,189.92862","492,074.13452","826,719,576,719,576.75000",25.60014,"26,632,036.48946","-7,949.71462",0.50000,"984,297.75688","984,148.26904",-149.48783,-123.88769,-0.00013,-0.00017,0.96389,"1,627,675.96651",-478.62634,1.43588,-0.00015,0.00003,1.94000
2022-10-01 06:00:00,buy,0.00000,0.00029,-0.00084,0.00029,0.00029,0.00000,0.00030,"1,678,657,001.25990","492,488.68357","826,719,576,719,576.75000",26.84753,"25,217,847.82075","-7,533.91818",0.50000,"985,112.80865","984,977.36714",-135.44150,-108.59397,-0.00011,-0.00008,0.97540,"-1,414,188.66872",415.79644,1.24739,-0.00014,0.00003,1.94000
